In [1]:
import mlflow

from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "http://localhost:5000"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("C")

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [2]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='5',    # Experiment ID we want
    #filter_string="metrics.recall > 0.7",
    filter_string="metrics.f1 > 0.3",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    #order_by=["metrics.recall ASC"]
    order_by=["metrics.f1 ASC"]
)

In [3]:
for run in runs:
    print(f"run id: {run.info.run_id}, f1: {run.data.metrics['f1']:.4f}")

run id: 0c4fec5fd752413ea9808a392ec087aa, f1: 0.3478
run id: de22006e68584f8b80632138d4ccd928, f1: 0.3478
run id: 533f3371b707417f8864efcd7d30d060, f1: 0.3478
run id: 0e308892254b405c90971e0aab2ef7aa, f1: 0.3478
run id: 04239d9115c645c683bb35054f9cd2a5, f1: 0.3478


In [4]:
runs[0].info.run_id

'0c4fec5fd752413ea9808a392ec087aa'

In [5]:
logged_model = 'runs:/' + runs[0].info.run_id + '/models_mlflow'

print(logged_model)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
mlflow.register_model(model_uri=logged_model, name="MRI_predictor")

runs:/0c4fec5fd752413ea9808a392ec087aa/models_mlflow


Registered model 'MRI_predictor' already exists. Creating a new version of this model...
2024/08/18 14:18:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MRI_predictor, version 5
Created version '5' of model 'MRI_predictor'.


<ModelVersion: aliases=[], creation_timestamp=1723990700897, current_stage='None', description='', last_updated_timestamp=1723990700897, name='MRI_predictor', run_id='0c4fec5fd752413ea9808a392ec087aa', run_link='', source='/workspaces/Testrepo/artifacts/5/0c4fec5fd752413ea9808a392ec087aa/artifacts/models_mlflow', status='READY', status_message='', tags={}, user_id='', version='5'>

In [9]:
with open('../pickle/model.pkl', 'wb') as f_out:
    pickle.dump(loaded_model, f_out)


## Code to test the model output:

In [7]:
import pickle
with open('../pickle/val.pkl', 'rb') as f_in:
    X_val,y_val = pickle.load(f_in)


In [ ]:
import pandas as pd
patient = {
    'm/f': 1,
    'age': 99,
    'educ':2.0,
    'ses': 0.0,
    'etiv': 1500,
    'nwbv': 0.1,    
}
X_patient = pd.DataFrame.from_dict([patient])
X_patient

In [ ]:
import numpy as np
# Predict on a Pandas DataFrame.
y_pred = loaded_model.predict(X_patient)

dementia_threshold = np.log1p(1.5 / 26)
y_pred = (y_pred > dementia_threshold).astype(int)
print(y_pred, y_val.to_numpy())